# Overview

The purpose of this notebook is to help with feature selection. Specifically I have calculated team stats using 1 ,3, 5, 10, 20, 30, and 40, 50 game rolling windows and using the raw totals vs scoring and venue adjustments. I want to determine which rolling windows or mix of windows results in the best model. To do so I will first train a simple logistic regression model using each rolling window and a mix. Then I will try Scikit-learn's select K best to see one comes out from that. 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, FunctionTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import make_pipeline, Pipeline
pd.set_option('display.max_columns', None)

In [74]:
#define function to model and evaluate on a simple logistic regression
def model_and_evaluate(df, feature_columns, scoring = 'neg_log_loss'):
    #for final modeling, train on 2017 to 2020 and test on 2021 season. So here I will look at CV scores only on the 2017-2020 season to avoid any data leakage when evalutating feature options
    X_train = df[df['Season'] != '2020-2021'].dropna()
    y_train = df[df['Season'] != '2020-2021'].dropna()['Home_Team_Won']
    
    numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

    categorical_features = ['B2B_Status']

    categorical_transformer = Pipeline(steps=[
        ('ohe', OneHotEncoder())])
    
    numeric_features = feature_columns.copy()
    numeric_features.remove('B2B_Status')
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('logisticregression', LogisticRegression(random_state =2020, max_iter=10000))])



    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring = scoring).mean()


# Evaluate Features With No Scoring and Venue Adjustments

In [7]:
df = pd.read_csv('data/all_games_multirolling_noSVA.csv')

In [8]:
df.head()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,away_Team_Key,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk
0,0,2017020001,2017-10-04,Bell MTS Place,WPG,TOR,2017-10-04 23:00:00,2,7,Final,0,WPG_2017-10-04,TOR_2017-10-04,Steve Mason,0.946716,0.259293,0.917127,Frederik Andersen,0.947245,0.043154,0.874317,WPG_2017-10-04,50.222222,60.000000,46.666667,10.112360,3.444444,0.063871,4.005556,0.090707,0.0,52.560647,56.521739,51.729107,9.285714,4.190000,0.079714,3.693333,0.098556,51.240876,50.000000,51.893740,7.228916,5.128333,0.091453,4.315000,0.128158,49.213483,51.190476,50.903164,8.977035,5.860833,0.102801,5.440000,0.105423,49.276086,50.000000,48.848563,8.720113,5.171111,0.119961,4.815000,0.115057,TOR_2017-10-04,53.777778,50.000000,51.673469,8.823529,6.450000,0.106460,3.394444,0.111948,0.0,53.457447,47.368421,51.673469,7.086614,5.893333,0.069910,3.070000,0.074267,50.712251,48.648649,50.580307,7.860262,5.073333,0.080420,4.235000,0.114522,49.103448,48.611111,48.707092,7.526882,4.525833,0.118431,4.195000,0.123361,50.022655,50.925926,50.175861,7.462687,4.538889,0.128519,4.363889,0.133978
1,1,2017020002,2017-10-04,PPG Paints Arena,PIT,STL,2017-10-05 00:00:00,4,5,Final,0,PIT_2017-10-04,STL_2017-10-04,Matt Murray,0.944706,0.457950,0.889831,Jake Allen,0.952147,0.199829,0.898810,PIT_2017-10-04,42.180095,50.000000,46.913580,13.559322,2.761111,0.182294,3.683333,0.171041,0.0,40.960452,57.142857,45.309026,12.121212,3.336667,0.143856,3.546667,0.153383,43.557423,45.238095,43.044560,8.558559,3.633333,0.176422,3.358333,0.113151,47.997139,52.439024,48.360562,8.811475,4.792500,0.134168,4.569167,0.119825,49.062049,52.991453,50.927977,8.310992,4.693889,0.123707,4.812778,0.120859,STL_2017-10-04,43.661972,47.058824,47.543860,10.666667,7.283333,0.113043,5.250000,0.067937,0.0,46.290801,50.000000,50.637826,11.570248,6.000000,0.096000,4.966667,0.109128,50.678733,60.465116,55.805893,10.116732,5.313333

In [9]:
list(df.columns)

['Unnamed: 0',
 'game_id',
 'date',
 'venue',
 'home_team',
 'away_team',
 'start_time',
 'home_score',
 'away_score',
 'status',
 'Home_Team_Won',
 'Home_Team_Key',
 'Away_Team_Key',
 'home_goalie',
 'home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_goalie',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%',
 'home_Team_Key',
 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk',
 'home_B2B',
 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk',
 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'home_la

In [10]:
#define different feature rolling game sets
r3 = ['home_B2B', 'away_B2B', 'home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk', 'away_last_3_FF%_5v5',
 'away_last_3_GF%_5v5',
 'away_last_3_xGF%_5v5',
 'away_last_3_SH%',
 'away_last3_pp_TOI_per_game',
 'away_last3_xGF_per_min_pp',
 'away_last3_pk_TOI_per_game',
 'away_last3_xGA_per_min_pk']
r5 =['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk', 'away_last_5_FF%_5v5',
 'away_last_5_GF%_5v5',
 'away_last_5_xGF%_5v5',
 'away_last_5_SH%',
 'away_last5_pp_TOI_per_game',
 'away_last5_xGF_per_min_pp',
 'away_last5_pk_TOI_per_game',
 'away_last5_xGA_per_min_pk']
r10 =['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'home_last10_pk_TOI_per_game',
 'home_last10_xGA_per_min_pk', 'away_last_10_FF%_5v5',
 'away_last_10_GF%_5v5',
 'away_last_10_xGF%_5v5',
 'away_last_10_SH%',
 'away_last10_pp_TOI_per_game',
 'away_last10_xGF_per_min_pp',
 'away_last10_pk_TOI_per_game',
 'away_last10_xGA_per_min_pk']
r20 = ['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_20_FF%_5v5',
 'home_last_20_GF%_5v5',
 'home_last_20_xGF%_5v5',
 'home_last_20_SH%',
 'home_last20_pp_TOI_per_game',
 'home_last20_xGF_per_min_pp',
 'home_last20_pk_TOI_per_game',
 'home_last20_xGA_per_min_pk', 'away_last_20_FF%_5v5',
 'away_last_20_GF%_5v5',
 'away_last_20_xGF%_5v5',
 'away_last_20_SH%',
 'away_last20_pp_TOI_per_game',
 'away_last20_xGF_per_min_pp',
 'away_last20_pk_TOI_per_game',
 'away_last20_xGA_per_min_pk']
r30 = ['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B',  'home_last_30_FF%_5v5',
 'home_last_30_GF%_5v5',
 'home_last_30_xGF%_5v5',
 'home_last_30_SH%',
 'home_last30_pp_TOI_per_game',
 'home_last30_xGF_per_min_pp',
 'home_last30_pk_TOI_per_game',
 'home_last30_xGA_per_min_pk', 'away_last_30_FF%_5v5',
 'away_last_30_GF%_5v5',
 'away_last_30_xGF%_5v5',
 'away_last_30_SH%',
 'away_last30_pp_TOI_per_game',
 'away_last30_xGF_per_min_pp',
 'away_last30_pk_TOI_per_game',
 'away_last30_xGA_per_min_pk']

all_r = list(set(r3+r5+r10+r20+r30))

r3_30 =list(set(r3+r30))
r5_30 = list(set(r5+r30))
r10_30 = list(set(r10+r30))
r_3_5_30 = list(set(r3+r5+r30))
r_5_20 = list(set(r5+r20))

In [11]:
windows =  {'3' : r3,'5' : r5,'10' : r10,'20' : r20, '30' : r30, 'all' : all_r, '3+30': r3_30, '5+30': r5_30, '10+30' : r10_30, '3+5+30': r_3_5_30, '5+20': r_5_20}

I can see from the below results that longer rolling game windows are scoring better. Using 5 games + the longer window shows some potential

In [12]:
for k,v in windows.items():
    print(f'{k} rolling cv log loss = {model_and_evaluate(df, v)}')

3 rolling cv log loss = -0.6861562495735097
5 rolling cv log loss = -0.6847321956176653
10 rolling cv log loss = -0.6809962171610652
20 rolling cv log loss = -0.6771632184554629
30 rolling cv log loss = -0.674385744275857
all rolling cv log loss = -0.6862683128672741
3+30 rolling cv log loss = -0.6776354571333818
5+30 rolling cv log loss = -0.6771719023613585
10+30 rolling cv log loss = -0.6788676522172316
3+5+30 rolling cv log loss = -0.6811070794335059
5+20 rolling cv log loss = -0.6807867901085245


In [14]:
for k,v in windows.items():
    print(f'{k} rolling cv accuracy = {model_and_evaluate(df, v, "accuracy")}')

3 rolling cv accuracy = 0.5573552779464389
5 rolling cv accuracy = 0.5579314532277442
10 rolling cv accuracy = 0.5687883524745823
20 rolling cv accuracy = 0.575533651134049
30 rolling cv accuracy = 0.5802321926096262
all rolling cv accuracy = 0.5664423114188638
3+30 rolling cv accuracy = 0.5849285809638232
5+30 rolling cv accuracy = 0.581990000904311
10+30 rolling cv accuracy = 0.5778865821781837
3+5+30 rolling cv accuracy = 0.5737792878335723
5+20 rolling cv accuracy = 0.5687909362202385


## Using Select K Best

In [28]:
X = df[all_r]
y= df['Home_Team_Won']

In [15]:
df['home_B2B'] = df['home_B2B'].astype('category')
df['away_B2B'] = df['away_B2B'].astype('category')

In [29]:
numeric_features = [
 'away_last3_xGF_per_min_pp',
 'home_last20_pk_TOI_per_game',
 'away_last_5_SH%',
 'away_last_10_SH%',
 'home_last_10_xGF%_5v5',
 'home_last20_xGF_per_min_pp',
 'home_last_3_xGF%_5v5',
 'home_last3_pp_TOI_per_game',
 'away_last30_xGA_per_min_pk',
 'home_Last_20_HDCSV%',
 'home_last30_pk_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'away_last_3_xGF%_5v5',
 'home_Last_20_FenwickSV%',
 'away_Last_20_HDCSV%',
 'away_last_30_xGF%_5v5',
 'away_last20_pk_TOI_per_game',
 'home_last30_xGF_per_min_pp',
 'home_last_20_xGF%_5v5',
 'away_last5_xGF_per_min_pp',
 'away_last10_pk_TOI_per_game',
 'home_last_20_SH%',
 'home_last_3_FF%_5v5',
 'home_last10_pk_TOI_per_game',
 'away_last_30_SH%',
 'away_Last_20_GSAx/60',
 'home_last_10_SH%',
 'away_last5_pp_TOI_per_game',
 'home_last_30_GF%_5v5',
 'away_last_3_GF%_5v5',
 'home_last_5_GF%_5v5',
 'away_last30_pp_TOI_per_game',
 'home_last5_pp_TOI_per_game',
 'home_last10_xGA_per_min_pk',
 'away_last_20_GF%_5v5',
 'home_last3_xGA_per_min_pk',
 'home_last_10_GF%_5v5',
 'away_last_5_GF%_5v5',
 'home_last_30_SH%',
 'home_last_30_FF%_5v5',
 'home_Last_20_GSAx/60',
 'home_last5_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last20_xGA_per_min_pk',
 'away_last_3_SH%',
 'away_last_20_FF%_5v5',
 'away_last30_pk_TOI_per_game',
 'away_last_5_FF%_5v5',
 'home_last20_pp_TOI_per_game',
 'home_last_30_xGF%_5v5',
 'home_last_5_FF%_5v5',
 'home_last_5_xGF%_5v5',
 'away_last_10_xGF%_5v5',
 'home_last_10_FF%_5v5',
 'away_last_3_FF%_5v5',
 'home_last10_pp_TOI_per_game',
 'home_last30_pp_TOI_per_game',
 'home_last_3_SH%',
 'away_last3_pp_TOI_per_game',
 'home_last5_xGA_per_min_pk',
 'away_last_20_SH%',
 'away_last_30_FF%_5v5',
 'away_last10_xGA_per_min_pk',
 'home_last_3_GF%_5v5',
 'away_last30_xGF_per_min_pp',
 'home_last_20_GF%_5v5',
 'away_last5_xGA_per_min_pk',
 'home_last_5_SH%',
 'away_last_20_xGF%_5v5',
 'away_last_5_xGF%_5v5',
 'home_last_20_FF%_5v5',
 'away_Last_20_FenwickSV%',
 'away_last5_pk_TOI_per_game',
 'away_last10_xGF_per_min_pp',
 'away_last3_xGA_per_min_pk',
 'away_last20_pp_TOI_per_game',
 'away_last10_pp_TOI_per_game',
 'away_last_10_GF%_5v5',
 'away_last_30_GF%_5v5',
 'home_last5_pk_TOI_per_game',
 'away_last_10_FF%_5v5',
 'home_last30_xGA_per_min_pk',
 'away_last20_xGF_per_min_pp',
 'away_last20_xGA_per_min_pk',
 'away_last3_pk_TOI_per_game',
 'home_last3_xGF_per_min_pp']

In [44]:

#this code worked i think
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# categorical_features = ['home_B2B', 'away_B2B']

# categorical_transformer = Pipeline(steps=[
#     ('ohe', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
 ])

skb_pipeline = Pipeline(steps=[('ss', StandardScaler()),
                      ('skb', SelectKBest(k = 20))])

In [34]:
X.head()

,away_last_20_xGF%_5v5,home_last_5_SH%,away_last_5_xGF%_5v5,away_last5_xGA_per_min_pk,home_last_3_FF%_5v5,away_last_5_FF%_5v5,home_last5_pk_TOI_per_game,away_last_3_FF%_5v5,home_last10_xGA_per_min_pk,home_last_20_xGF%_5v5,home_last_10_SH%,away_last3_pp_TOI_per_game,away_Last_20_FenwickSV%,home_last10_xGF_per_min_pp,home_last10_pk_TOI_per_game,away_last30_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last_30_GF%_5v5,away_last_30_SH%,home_last_5_GF%_5v5,away_last3_pk_TOI_per_game,home_last_5_xGF%_5v5,away_last10_pk_TOI_per_game,home_last20_xGA_per_min_pk,away_last_30_FF%_5v5,home_Last_20_HDCSV%,home_last5_pp_TOI_per_game,away_last_3_GF%_5v5,away_last20_xGA_per_min_pk,home_last30_xGF_per_min_pp,away_last_5_SH%,away_last_30_xGF%_5v5,home_Last_20_FenwickSV%,away_B2B,away_last10_xGF_per_min_pp,home_last3_pk_TOI_per_game,away_last_10_GF%_5v5,away_last20_xGF_per_min_pp,home_last_3_GF%_5v5,home_last_30_SH%,away_last30_xGF_per_min_pp,home_last_20_FF%_5v5,away_last5_pp_TOI_per_game,home_Last_20_GSAx/60,away_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last20_pk_TOI_per_game,away_last_3_xGF%_5v5,away_last_3_SH%,away_last_10_xGF%_5v5,away_last30_pp_TOI_per_game,away_last20_pk_TOI_per_game,home_last30_xGA_per_min_pk,home_last_5_FF%_5v5,away_last_20_GF%_5v5,away_last_5_GF%_5v5,home_last_3_SH%,home_last_10_FF%_5v5,home_last_30_xGF%_5v5,away_last3_xGF_per_min_pp,home_last_20_GF%_5v5,home_last30_pk_TOI_per_game,home_last5_xGF_per_min_pp,home_last_30_FF%_5v5,home_last20_xGF_per_min_pp,home_B2B,away_last20_pp_TOI_per_game,away_last10_pp_TOI_per_game,away_last5_pk_TOI_per_game,away_Last_20_HDCSV%,home_last3_xGA_per_min_pk,home_last30_pp_TOI_per_game,away_last_20_FF%_5v5,home_last10_pp_TOI_per_game,away_last30_xGA_per_min_pk,home_last_3_xGF%_5v5,home_last20_pp_TOI_per_game,home_last3_pp_TOI_per_game,home_last_10_xGF%_5v5,away_Last_20_GSAx/60,home_last3_xGF_per_min_pp,away_last_10_SH%,away_last5_xGF_per_min_pp,home_last_20_SH%,home_last5_xGA_per_min_pk,away_last_20_SH%,home_last_30_GF%_5v5,away_last3_xGA_per_min_pk
0,48.707092,9.285714,51.673469,0.074267,50.222222,53.457447,3.693333,53.777778,0.128158,50.903164,7.228916,6.450000,0.947245,0.091453,4.315000,4.363889,0.114522,50.925926,7.462687,56.521739,3.394444,51.729107,4.235000,0.105423,50.022655,0.917127,4.190000,50.000000,0.123361,0.119961,7.086614,50.175861,0.946716,0.0,0.080420,4.005556,48.648649,0.118431,60.000000,8.720113,0.128519,49.213483,5.893333,0.259293,50.712251,50.000000,5.440000,51.673469,8.823529,50.580307,4.538889,4.195000,0.115057,52.560647,48.611111,47.368421,10.112360,51.240876,48.848563,0.106460,51.190476,4.815000,0.079714,49.276086,0.102801,0.0,4.525833,5.073333,3.070000,0.874317,0.090707,5.171111,49.103448,5.128333,0.133978,46.666667,5.860833,3.444444,51.893740,0.043154,0.063871,7.860262,0.069910,8.977035,0.098556,7.526882,50.000000,0.111948
1,51.396973,12.121212,50.637826,0.109128,42.180095,46.290801,3.546667,43.661972,0.113151,48.360562,8.558559,7.283333,0.952147,0.176422,3.358333,4.832222,0.088571,61.855670,8.196721,57.142857,5.250000,45.309026,4.900000,0.119825,48.991935,0.889831,3.336667,47.058824,0.079140,0.123707,11.570248,52.271593,0.944706,0.0,0.101255,3.683333,60.465116,0.102079,50.000000,8.310992,0.096376,47.997139,6.000000,0.457950,50.678733,45.238095,4.569167,47.543860,10.666667,55.805893,4.613889,5.041667,0.120859,40.960452,64.705882,50.000000,13.559322,43.557423,50.927977,0.113043,52.439024,4.812778,0.143856,49.062049,0.134168,0.0,4.810000,5.313333,4.966667,0.898810,0.171041,4.693889,48.849295,3.633333,0.087882,46.913580,4.792500,2.761111,43.044560,0.199829,0.182294,10.116732,0.096000,8.811475,0.153383,8.835341,52.991453,0.067937
2,45.886850,8.571429,39.044289,0.112415,60.106383,42.500000,4.540000,42.307692,0.132087,53.204661,10.317460,5.316667,0.941538,0.114090,4.815000,4.905000,0.101880,54.385965,9.158050,57.142857,5.644444,58.893871,5.585000,0.128953,49.484004,0.887892,6.283333,45.454545,0.099227,0.126840,9.375000,47.297977,0.937824,0.0,0.111983,5.166667,40.54

In [45]:

X = k_best_df[all_r]

y = k_best_df['Home_Team_Won']

In [46]:
skb_pipeline.fit(X,y)

Pipeline(steps=[('ss', StandardScaler()), ('skb', SelectKBest(k=20))])

In [47]:
cols = skb_pipeline.named_steps['skb'].get_support()
k_best_features = X.iloc[:,cols]

In [48]:
k = [n[9:] for n in k_best_features.columns]

The Select K best output is showing that GF%, xGF%, FF%, and GSAx/60 are most important. Since it is showing a mix of 10, 20 and 30 game rolling windows it is not helpful it narrowing the features down from that perspective

In [49]:
sorted(k)

['_10_FF%_5v5',
 '_10_FF%_5v5',
 '_10_GF%_5v5',
 '_10_xGF%_5v5',
 '_10_xGF%_5v5',
 '_20_FF%_5v5',
 '_20_FF%_5v5',
 '_20_FenwickSV%',
 '_20_GF%_5v5',
 '_20_GF%_5v5',
 '_20_GSAx/60',
 '_20_xGF%_5v5',
 '_20_xGF%_5v5',
 '_30_FF%_5v5',
 '_30_FF%_5v5',
 '_30_GF%_5v5',
 '_30_GF%_5v5',
 '_30_xGF%_5v5',
 '_30_xGF%_5v5',
 '_5_FF%_5v5']

# Evaluate With Venue and Scoring Adjustments

In [84]:
sva = pd.read_csv('/Users/gschwaeber/Documents/Data_Science/Capstone/data/all_games_multirolling_SVA_3.csv')

In [9]:
sva.head()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Goalie_FenwickSV%,home_Goalie_GSAx/60,home_Goalie_HDCSV%,away_goalie,away_Goalie_FenwickSV%,away_Goalie_GSAx/60,away_Goalie_HDCSV%,home_Game_Number,home_Team_Key,home_last_1_FF%_5v5,home_last_1_GF%_5v5,home_last_1_xGF%_5v5,home_last_1_SH%,home_last1_pp_TOI_per_game,home_last1_xGF_per_min_pp,home_last1_GF_per_min_pp,home_last1_pk_TOI_per_game,home_last1_xGA_per_min_pk,home_last1_GA_per_min_pk,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_GF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_last3_GA_per_min_pk,home_Last_Game_Date,home_Days_Since_Last_Game,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_GF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last5_GA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_GF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last10_GA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_GF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last20_GA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_GF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,home_last30_GA_per_min_pk,home_last_40_FF%_5v5,home_last_40_GF%_5v5,home_last_40_xGF%_5v5,home_last_40_SH%,home_last40_pp_TOI_per_game,home_last40_xGF_per_min_pp,home_last40_GF_per_min_pp,home_last40_pk_TOI_per_game,home_last40_xGA_per_min_pk,home_last40_GA_per_min_pk,home_last40_pp_TOI_per_game.1,home_last40_xGF_per_min_pp.1,home_last40_GF_per_min_pp.1,home_last40_pk_TOI_per_game.1,home_last40_xGA_per_min_pk.1,home_last40_GA_per_min_pk.1,home_last_50_FF%_5v5,home_last_50_GF%_5v5,home_last_50_xGF%_5v5,home_last_50_SH%,home_last50_pp_TOI_per_game,home_last50_xGF_per_min_pp,home_last50_GF_per_min_pp,home_last50_pk_TOI_per_game,home_last50_xGA_per_min_pk,home_last50_GA_per_min_pk,away_Game_Number,away_Team_Key,away_last_1_FF%_5v5,away_last_1_GF%_5v5,away_last_1_xGF%_5v5,away_last_1_SH%,away_last1_pp_TOI_per_game,away_last1_xGF_per_min_pp,away_last1_GF_per_min_pp,away_last1_pk_TOI_per_game,away_last1_xGA_per_min_pk,away_last1_GA_per_min_pk,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_GF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_last3_GA_per_min_pk,away_Last_Game_Date,away_Days_Since_Last_Game,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_GF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last5_GA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_GF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last10_GA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_GF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last20_GA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_GF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk,away_last30_GA_per_min_pk,away_last_40_FF%_5v5,away_last_

In [10]:
list(sva.columns)

['Unnamed: 0',
 'game_id',
 'date',
 'venue',
 'home_team',
 'away_team',
 'start_time',
 'home_score',
 'away_score',
 'status',
 'Home_Team_Won',
 'Home_Team_Key',
 'Away_Team_Key',
 'home_goalie',
 'home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_goalie',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%',
 'home_Game_Number',
 'home_Team_Key',
 'home_last_1_FF%_5v5',
 'home_last_1_GF%_5v5',
 'home_last_1_xGF%_5v5',
 'home_last_1_SH%',
 'home_last1_pp_TOI_per_game',
 'home_last1_xGF_per_min_pp',
 'home_last1_GF_per_min_pp',
 'home_last1_pk_TOI_per_game',
 'home_last1_xGA_per_min_pk',
 'home_last1_GA_per_min_pk',
 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_GF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk',
 'home_last3_GA_per_min_pk',
 'home_Last_Game_Date',
 'home_Days_Since_Last_Game

In [76]:
common = ['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 
 'home_Rating.A.Pre',
 'away_Rating.A.Pre',
 'B2B_Status']

r1 = ['home_last_1_FF%_5v5',
 'home_last_1_GF%_5v5',
 'home_last_1_xGF%_5v5',
 'home_last_1_SH%',
 'home_last1_xGF_per_min_pp',
 'home_last1_GF_per_min_pp',
 'home_last1_xGA_per_min_pk',
 'home_last1_GA_per_min_pk',
 'away_last_1_FF%_5v5',
 'away_last_1_GF%_5v5',
 'away_last_1_xGF%_5v5',
 'away_last_1_SH%',
 'away_last1_xGF_per_min_pp',
 'away_last1_GF_per_min_pp',
 'away_last1_xGA_per_min_pk',
 'away_last1_GA_per_min_pk'] + common

r3 = ['home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_xGF_per_min_pp',
 'home_last3_GF_per_min_pp',
 'home_last3_xGA_per_min_pk',
 'home_last3_GA_per_min_pk',
 'away_last_3_FF%_5v5',
 'away_last_3_GF%_5v5',
 'away_last_3_xGF%_5v5',
 'away_last_3_SH%',
 'away_last3_xGF_per_min_pp',
 'away_last3_GF_per_min_pp',
 'away_last3_xGA_per_min_pk',
 'away_last3_GA_per_min_pk'] + common

r5 =['home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_xGF_per_min_pp',
 'home_last5_GF_per_min_pp',
 'home_last5_xGA_per_min_pk',
 'home_last5_GA_per_min_pk',
 'away_last_5_FF%_5v5',
 'away_last_5_GF%_5v5',
 'away_last_5_xGF%_5v5',
 'away_last_5_SH%',
 'away_last5_xGF_per_min_pp',
 'away_last5_GF_per_min_pp',
 'away_last5_xGA_per_min_pk',
 'away_last5_GA_per_min_pk'] + common

r10 =['home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_xGF_per_min_pp',
 'home_last10_GF_per_min_pp',
 'home_last10_xGA_per_min_pk',
 'home_last10_GA_per_min_pk',
  'away_last_10_FF%_5v5',
 'away_last_10_GF%_5v5',
 'away_last_10_xGF%_5v5',
 'away_last_10_SH%',
 'away_last10_xGF_per_min_pp',
 'away_last10_GF_per_min_pp',
 'away_last10_xGA_per_min_pk',
 'away_last10_GA_per_min_pk'] + common


r20 = ['home_last_20_FF%_5v5',
 'home_last_20_GF%_5v5',
 'home_last_20_xGF%_5v5',
 'home_last_20_SH%',
 'home_last20_xGF_per_min_pp',
 'home_last20_GF_per_min_pp',
 'home_last20_xGA_per_min_pk',
 'home_last20_GA_per_min_pk',
 'away_last_20_FF%_5v5',
 'away_last_20_GF%_5v5',
 'away_last_20_xGF%_5v5',
 'away_last_20_SH%',
 'away_last20_xGF_per_min_pp',
 'away_last20_GF_per_min_pp',
 'away_last20_xGA_per_min_pk',
 'away_last20_GA_per_min_pk'] +common

r30 = ['home_last_30_FF%_5v5',
 'home_last_30_GF%_5v5',
 'home_last_30_xGF%_5v5',
 'home_last_30_SH%',
 'home_last30_xGF_per_min_pp',
 'home_last30_GF_per_min_pp',
 'home_last30_xGA_per_min_pk',
 'home_last30_GA_per_min_pk',
 'away_last_30_FF%_5v5',
 'away_last_30_GF%_5v5',
 'away_last_30_xGF%_5v5',
 'away_last_30_SH%',
 'away_last30_xGF_per_min_pp',
 'away_last30_GF_per_min_pp',
 'away_last30_xGA_per_min_pk',
 'away_last30_GA_per_min_pk'] + common


r40 = ['home_last_40_FF%_5v5',
 'home_last_40_GF%_5v5',
 'home_last_40_xGF%_5v5',
 'home_last_40_SH%',
 'home_last40_xGF_per_min_pp',
 'home_last40_GF_per_min_pp',
 'home_last40_xGA_per_min_pk',
 'home_last40_GA_per_min_pk',
 'away_last_40_FF%_5v5',
 'away_last_40_GF%_5v5',
 'away_last_40_xGF%_5v5',
 'away_last_40_SH%',
 'away_last40_xGF_per_min_pp',
 'away_last40_GF_per_min_pp',
 'away_last40_xGA_per_min_pk',
 'away_last40_GA_per_min_pk'] + common

r50 = ['home_last_50_FF%_5v5',
 'home_last_50_GF%_5v5',
 'home_last_50_xGF%_5v5',
 'home_last_50_SH%',
 'home_last50_xGF_per_min_pp',
 'home_last50_GF_per_min_pp',
 'home_last50_xGA_per_min_pk',
 'home_last50_GA_per_min_pk',
 'away_last_50_FF%_5v5',
 'away_last_50_GF%_5v5',
 'away_last_50_xGF%_5v5',
 'away_last_50_SH%',
 'away_last50_xGF_per_min_pp',
 'away_last50_GF_per_min_pp',
 'away_last50_xGA_per_min_pk',
 'away_last50_GA_per_min_pk'] + common


all_r = list(set(r3+r5+r10+r20+r30))

r3_30 =list(set(r3+r30))
r5_30 = list(set(r5+r30))
r10_30 = list(set(r10+r30))
r_3_5_30 = list(set(r3+r5+r30))
r_5_20 = list(set(r5+r20))
r_5_40 = list(set(r5+r40))

In [77]:
windows =  {'1': r1, '3' : r3,'5' : r5,'10' : r10,'20' : r20, '30' : r30, '40' : r40, '50': r50, 'all' : all_r, '3+30': r3_30, '5+30': r5_30, '10+30' : r10_30, '3+5+30': r_3_5_30, '5+20': r_5_20, '5+40' : r_5_40 }

After evaluating the data with no scoring and venue adjustment(SVA), I added a 40 game rolling window to review with the SVA data. The 40 game window is the most promising with the lowest log loss. The 5 and 40 combined windows also show promise. Additionally comparing the 30 game window from the raw vs the SVA data, the SVA data has a lower log loss and appeart to be superior.

In [70]:
#checking for infinite values and dropping
pd.set_option("display.max_rows", 20)
sva1 = sva.dropna(subset = ['home_Rating.A.Pre'])

r = sva1[r1].index[np.isinf(sva1[r1]).any(1)]

r

Int64Index([ 161,  308,  346,  399,  450,  451,  650,  872,  940, 1075, 1376,
            1608, 1768, 2008, 2223, 2244, 2265, 2410, 2461, 2535, 2548, 2728,
            2938, 3053, 3236, 3469, 3518, 3867, 3877, 3914, 3922, 4054, 4173,
            4174, 4177, 4371],
           dtype='int64')

In [71]:
sva1 = sva1.drop(index=r)

In [78]:
for k,v in windows.items():
    print(f'{k} rolling cv log loss = {model_and_evaluate(sva1, v)}')

1 rolling cv log loss = -0.681743840701009
3 rolling cv log loss = -0.6824750160737495
5 rolling cv log loss = -0.6806028925591109
10 rolling cv log loss = -0.6790897151577344
20 rolling cv log loss = -0.680232903596563
30 rolling cv log loss = -0.6803889424949425
40 rolling cv log loss = -0.6786612628646052
50 rolling cv log loss = -0.679133616838792
all rolling cv log loss = -0.6993469447370013
3+30 rolling cv log loss = -0.6867413175473175
5+30 rolling cv log loss = -0.6849884213438509
10+30 rolling cv log loss = -0.6854996362496376
3+5+30 rolling cv log loss = -0.6904086232018225
5+20 rolling cv log loss = -0.6852509541316462
5+40 rolling cv log loss = -0.6827305174117813


In [79]:
for k,v in windows.items():
    print(f'{k} rolling cv accuracy = {model_and_evaluate(sva1, v, "accuracy")}')

1 rolling cv accuracy = 0.5711180124223602
3 rolling cv accuracy = 0.5680124223602485
5 rolling cv accuracy = 0.5686335403726708
10 rolling cv accuracy = 0.5754658385093168
20 rolling cv accuracy = 0.5686335403726708
30 rolling cv accuracy = 0.567391304347826
40 rolling cv accuracy = 0.5711180124223602
50 rolling cv accuracy = 0.5720496894409939
all rolling cv accuracy = 0.5484472049689442
3+30 rolling cv accuracy = 0.5636645962732919
5+30 rolling cv accuracy = 0.5636645962732919
10+30 rolling cv accuracy = 0.5642857142857143
3+5+30 rolling cv accuracy = 0.5531055900621118
5+20 rolling cv accuracy = 0.5546583850931677
5+40 rolling cv accuracy = 0.5664596273291925


# Conclusion

Using Venue and Scoring adjustment for the 5v5 data looks promising. It also seems that the more rolling games, the more predictive the features are, however log loss diminishes after 40 . Including a mix of 5 and 40 game rolling features could provide the best mix of capturing season long team evaluation with short term streakiness. When modeling I will try using 40 games only, 5 and 40, and trying recursive feature elimiation on all windows to see if that algorithm can find patternes within the feature set.